In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

**Loading Path**

In [2]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

In [3]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [4]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

merged_df["steamengines_number"] = merged_df["steamengines_number"]
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [8]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

In [9]:
merged_df["per_unskilled"] = merged_df['per_unskilled']*100
merged_df["per_low"] = merged_df['per_low']*100
merged_df["per_medium"] = merged_df['per_medium']*100
merged_df["per_high"] = merged_df['per_high']*100

In [10]:
log_list = ["log_distance_carbon_c", 
                     "s_industry",
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km",
                     "uni_dummy", 
                     "min_distance_uni", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high"]

for col in log_list:
    merged_df[col] = merged_df[col]+1
    merged_df[col] = np.log(merged_df[col])

# 2SLS
Lets run a good'ol reliable IV first

## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [11]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [12]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_industry",
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km",
                     "uni_dummy", 
                     "min_distance_uni", 
                     "prov_cap_dummy", 
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "horsepower"]]

In [13]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud",
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "BER_dummy"]

control_list_2 = ["soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "BER_dummy"]

In [14]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var,
                                                          "log_distance_carbon_c",
                                                          "steamengines_number",
                                                          control_list,
                                                          "per_unskilled")

first_low, second_low, res_low = twoSLS(dep_var,
                                        "log_distance_carbon_c",
                                        "steamengines_number",
                                        control_list,
                                        "per_low")

first_medium, second_medium, res_medium = twoSLS(dep_var,
                                                 "log_distance_carbon_c",
                                                 "steamengines_number",
                                                 control_list,
                                                 "per_medium")

first_high, second_high, res_high = twoSLS(dep_var,
                                           "log_distance_carbon_c",
                                           "steamengines_number",
                                           control_list,
                                           "per_high")

In [15]:
first_unskilled[[0,6,8,12]]

,0,6,8,12
Intercept,2.906*** (0.069),-29.722*** (3.710),-25.785*** (4.162),-20.337*** (6.020)
log_distance_carbon_c,-0.280*** (0.044),-0.440*** (0.050),-0.417*** (0.052),-0.416*** (0.051)
Adj. R-squared,0.063,0.248,0.251,0.299
F-statistic,39.510,28.050,22.411,19.842
longitud,NaN,0.594*** (0.118),0.570*** (0.125),0.605*** (0.136)
latitud,NaN,5.728*** (0.919),4.719*** (1.058),3.343** (1.437)
soil_suitability,NaN,-0.433*** (0.072),-0.430*** (0.072),-0.342*** (0.072)
mean_temp,NaN,3.000*** (0.345),2.965*** (0.348),3.260*** (0.341)
mean_preci,NaN,0.572*** (0.162),0.626*** (0.164),0.658*** (0.161)
dist_100km,NaN,0.072 (0.131),0.078 (0.132),0.056 (0.129)


In [16]:
# First Stage (same for every regression)
second_unskilled[[0,6,8,12]]

,0,6,8,12
Intercept,5.280*** (0.227),0.649 (2.259),0.639 (2.249),-2.023 (3.343)
predicted,-0.554*** (0.091),-0.266*** (0.068),-0.209*** (0.073),-0.198*** (0.068)
Adj. R-squared,0.059,0.185,0.217,0.332
std err,0.091059,0.067757,0.07251,0.067976
longitud,NaN,-0.061 (0.084),-0.205** (0.087),-0.130 (0.091)
latitud,NaN,1.599*** (0.481),2.061*** (0.514),2.526*** (0.765)
soil_suitability,NaN,-0.053 (0.053),-0.037 (0.053),0.018 (0.047)
mean_temp,NaN,-0.573* (0.296),-0.885*** (0.305),-0.736** (0.299)
mean_preci,NaN,-0.238** (0.111),-0.319*** (0.116),-0.348*** (0.109)
dist_100km,NaN,0.210*** (0.079),0.149* (0.078),0.069 (0.073)


In [17]:
second_low[[0,6,8,12]]

,0,6,8,12
Intercept,0.458* (0.250),19.006*** (2.373),17.148*** (2.371),15.297*** (3.709)
predicted,1.074*** (0.100),0.326*** (0.071),0.312*** (0.076),0.299*** (0.075)
Adj. R-squared,0.166,0.338,0.360,0.395
std err,0.099972,0.07118,0.076457,0.075422
longitud,NaN,0.012 (0.088),0.144 (0.092),0.117 (0.101)
latitud,NaN,-4.794*** (0.506),-4.681*** (0.542),-4.152*** (0.849)
soil_suitability,NaN,0.161*** (0.056),0.162*** (0.056),0.104** (0.052)
mean_temp,NaN,0.208 (0.311),0.411 (0.321),0.247 (0.331)
mean_preci,NaN,0.423*** (0.117),0.434*** (0.123),0.436*** (0.121)
dist_100km,NaN,-0.246*** (0.083),-0.195** (0.082),-0.139* (0.081)


In [18]:
second_medium[[0,6,8,12]]

,0,6,8,12
Intercept,2.926*** (0.162),-1.037 (1.607),0.254 (1.560),0.061 (2.385)
predicted,0.134** (0.065),0.166*** (0.048),0.086* (0.050),0.070 (0.049)
Adj. R-squared,0.006,0.135,0.211,0.288
std err,0.064682,0.048219,0.050294,0.048503
longitud,NaN,-0.126** (0.060),0.008 (0.061),0.004 (0.065)
latitud,NaN,1.049*** (0.343),0.246 (0.356),0.349 (0.546)
soil_suitability,NaN,-0.051 (0.038),-0.076** (0.037),-0.107*** (0.033)
mean_temp,NaN,0.161 (0.211),0.512** (0.211),0.465** (0.213)
mean_preci,NaN,-0.061 (0.079),0.059 (0.081),0.082 (0.078)
dist_100km,NaN,-0.092 (0.056),-0.031 (0.054),0.020 (0.052)


In [19]:
second_high[[0,6,8,12]]

,0,6,8,12
Intercept,-0.139 (0.195),1.423 (1.977),1.537 (1.976),5.147* (2.949)
predicted,0.297*** (0.078),0.266*** (0.059),0.219*** (0.064),0.213*** (0.060)
Adj. R-squared,0.023,0.119,0.148,0.267
std err,0.078152,0.059313,0.063692,0.05996
longitud,NaN,-0.118 (0.073),-0.005 (0.077),-0.080 (0.081)
latitud,NaN,-0.175 (0.421),-0.569 (0.451),-1.257* (0.675)
soil_suitability,NaN,0.097** (0.046),0.083* (0.047),0.031 (0.041)
mean_temp,NaN,0.015 (0.259),0.266 (0.268),0.142 (0.263)
mean_preci,NaN,-0.148 (0.098),-0.080 (0.102),-0.039 (0.096)
dist_100km,NaN,-0.198*** (0.069),-0.149** (0.069),-0.079 (0.064)
